# **Load Dataset**

jika dijalankan di gdrive import drive nya dulu lalu sesuaikan path nya

In [1]:
from google.colab import drive
drive.mount ('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Kuliah/Pemrograman Sistem Cerdas 1/Prak_Nur Rohman/Revisi Deployment Model Sentiment Analysis

/content/drive/MyDrive/Kuliah/Pemrograman Sistem Cerdas 1/Prak_Nur Rohman/Revisi Deployment Model Sentiment Analysis


# SCRAPING

In [7]:
%pip install Sastrawi
%pip install google-play-scraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.0 MB/s eta 0:00:00


In [8]:
from google_play_scraper import app
import pandas as pd
import numpy as np

In [ ]:
from google_play_scraper import Sort, reviews

result, continuation_token = reviews(
    'nozero.apps1', # ID = google play
    lang='id',
    country='id',
    sort=Sort.MOST_RELEVANT,
    count=200,
    filter_score_with= 1  # defaults to None(means all score) Use 1 or 2 or 3 or 4 or 5 to select certain score
)

In [ ]:
# Dataframe dengan nama
df_ulasan = pd.DataFrame(np.array(result),columns=['review'])
df_ulasan = df_ulasan.join(pd.DataFrame(df_ulasan.pop('review').tolist()))
df_ulasan.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,b9827abe-7e87-4701-a148-54b1c6771e85,Amanda Hani,https://play-lh.googleusercontent.com/a-/ALV-U...,Halo Riliv! Aplikasi kalian sebenarnya bagus. ...,1,21,3.6.1,2022-08-21 02:46:50,Hi Riliver Amanda. Terimakasih telah mendownlo...,2022-08-22 12:10:10,3.6.1
1,961f1de0-8566-4a2e-902b-fdfc63fe104b,Daylon Nevan,https://play-lh.googleusercontent.com/a-/ALV-U...,Parah nih gara2 ini aplikasi jadi masang alarm...,1,2,None,2021-09-28 14:00:29,"Hai, Riliver Daylon Nevan . Terimakasih telah ...",2021-09-30 03:24:41,None
2,b4a138ff-4020-45ba-aa2b-3ab418fc5ede,Jarvina Hwang,https://play-lh.googleusercontent.com/a-/ALV-U...,Ternyata banyak case yang sama. Setelah upgrad...,1,2,None,2022-02-23 00:28:30,"Hai, Riliver Jarvina Hwang. Apakah Anda masih ...",2022-04-04 03:00:42,None
3,81e8ca3c-55dc-422b-8d93-0f2403d33ce7,1PA01_NASRUL NOER HIDAYAT,https://play-lh.googleusercontent.com/a-/ALV-U...,Streak selalu tereset. Padahal saya setiap har...,1,2,3.7.8,2023-09-12 00:54:06,None,NaT,3.7.8
4,5a6ecb35-dde1-4880-a4ae-258f71898b80,Nur Aeni,https://play-lh.googleusercontent.com/a/ACg8oc...,Aplikasi ini sangat membantu saya untuk mengen...,1,1,3.7.8,2023-09-29 23:41:13,None,NaT,3.7.8


In [ ]:
df_ulasan[['content', 'score']].head()

,content,score
0,Halo Riliv! Aplikasi kalian sebenarnya bagus. ...,1
1,Parah nih gara2 ini aplikasi jadi masang alarm...,1
2,Ternyata banyak case yang sama. Setelah upgrad...,1
3,Streak selalu tereset. Padahal saya setiap har...,1
4,Aplikasi ini sangat membantu saya untuk mengen...,1


In [ ]:
my_ulasan = df_ulasan[['content', 'score']]

In [ ]:
my_ulasan.to_csv("riliv-rating1.csv", index = False)

# Penggabungan Data .CSV

In [ ]:
%ls

 kamus.csv       riliv-rating1.csv   riliv-rating5.csv                         svm_model.pkl
 ratingAll.csv   riliv-rating3.csv  'Sentimen Analisis Aplikasi Riliv.ipynb'


In [ ]:
df_rating5 = pd.read_csv("riliv-rating5.csv")
df_rating3 = pd.read_csv("riliv-rating3.csv")
df_rating1 = pd.read_csv("riliv-rating1.csv")


In [ ]:
df_ratingAll = pd.concat([df_rating5,df_rating3,df_rating1])

In [ ]:
len(df_ratingAll)

600

In [ ]:
df_ratingAll.to_csv("ratingAll.csv", index = False)

# Preprocessing

In [9]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier


# Load the dataset
dframe = pd.read_csv("ratingAll.csv")

## **Case Folding**

In [10]:
# Case Folding
data_casefolded = dframe['content'].str.lower()

## **Filtering**

In [11]:
# Filtering
def filter_text(text):
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)  # Remove URLs
    text = re.sub(r'\(cont\)', ' ', text)  # Remove continuations
    text = re.sub(r'[!"”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]', ' ', text)  # Remove punctuation
    text = re.sub(r'#\S+', '', text)  # Remove hashtags
    text = re.sub(r'\d+', ' ', text)  # Remove numbers
    return text

data_filtered = data_casefolded.apply(filter_text)

## **Tokenization**

In [12]:
# Tokenization
nltk.download('punkt')
data_tokenized = data_filtered.apply(word_tokenize)

print(data_tokenized)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


0      [sangat, seru, banyak, journal, journal, keren...
1      [saya, kira, mental, saya, butuh, penanganan, ...
2      [aplikasinya, sangat, membantu, saya, di, dala...
3      [aplikasi, yang, dapat, membantu, dalam, menja...
4      [aplikasi, ini, sangat, bagus, saya, ingin, be...
                             ...                        
595                    [apalah, baru, masuk, uda, bayar]
596                               [proses, login, gagal]
597                                      [lelet, apknya]
598                                 [berbayar, gak, sih]
599                                  [harus, ada, trial]
Name: content, Length: 600, dtype: object


## **Stopword Removal**

In [13]:
# Stopword Removal
factory = StopWordRemoverFactory()
stopword_list = factory.get_stop_words()

def remove_stopwords(tokens):
    return [word for word in tokens if word not in stopword_list and len(word) > 3]

data_no_stopwords = data_tokenized.apply(remove_stopwords)

print(data_no_stopwords)

0      [sangat, seru, banyak, journal, journal, keren...
1      [kira, mental, butuh, penanganan, mencoba, men...
2      [aplikasinya, sangat, membantu, aplikasi, bany...
3      [aplikasi, membantu, menjaga, kesehatan, menta...
4      [aplikasi, sangat, bagus, berterimakasiih, ban...
                             ...                        
595                         [apalah, baru, masuk, bayar]
596                               [proses, login, gagal]
597                                      [lelet, apknya]
598                                           [berbayar]
599                                              [trial]
Name: content, Length: 600, dtype: object


## **Slang Word Normalization**

In [14]:
path_dataslang = open("kamus.csv")
dataslang = pd.read_csv(path_dataslang, encoding = 'utf-8', header=None, sep=";")

def replaceSlang(word):
  if word in list(dataslang[0]):
    indexslang = list(dataslang[0]).index(word)
    return dataslang[1][indexslang]
  else:
    return word

ulasan_formal = []
for data in data_no_stopwords:
  data_clean = [replaceSlang(word) for word in data]
  ulasan_formal.append(data_clean)
len_ulasan_formal = len(ulasan_formal)
print(ulasan_formal)
len_ulasan_formal

[['sangat', 'seru', 'banyak', 'journal', 'journal', 'keren', 'bahasa', 'aplikasi', 'sangat', 'ringan', 'merasa', 'sedang', 'mengobrol', 'teman'], ['kira', 'mental', 'butuh', 'penanganan', 'mencoba', 'mencari', 'penanganan', 'aplikasi', 'sadar', 'penanganan', 'butuh', 'uang', 'mungkin', 'bertahan', 'beberapa', 'tahun', 'mendapatkan', 'penanganan', 'mental', 'terimakasih', 'sudah', 'membuat'], ['aplikasinya', 'sangat', 'membantu', 'aplikasi', 'banyak', 'ilmu', 'bermanfaat', 'tampilan', 'aplikasi', 'menarik', 'banyaknya', 'pilihan', 'pilih', 'kenyamanan', 'menggunakan', 'aplikasi'], ['aplikasi', 'membantu', 'menjaga', 'kesehatan', 'mental', 'memperbaiki', 'kesehatan', 'mental'], ['aplikasi', 'sangat', 'bagus', 'berterimakasiih', 'banyak', 'riliv', 'membantu', 'menemani', 'masa', 'masa', 'sulit'], ['sejauh', 'membantu', 'semoga', 'selalu', 'menemani', 'membuat', 'kesehatan', 'mental', 'membaik'], ['bagus', 'banget', 'banget', 'banget', 'semua', 'orang', 'ingin', 'membentuk', 'kebiasaan', '

600

## **Stemming**

In [16]:
factory = StemmerFactory()
ind_stemmer = factory.create_stemmer()
white_list = ["bali"]

def stemmer(line):
  valid = list()
  for word in line:
    if(word not in white_list):
      word = ind_stemmer.stem(word)
    if(len(word)>2):
      valid.append(word)
  return valid

data_stemmed = [stemmer (line) for line in ulasan_formal]
print(data_stemmed)

[['sangat', 'seru', 'banyak', 'journal', 'journal', 'keren', 'bahasa', 'aplikasi', 'sangat', 'ringan', 'rasa', 'sedang', 'obrol', 'teman'], ['kira', 'mental', 'butuh', 'tangan', 'coba', 'cari', 'tangan', 'aplikasi', 'sadar', 'tangan', 'butuh', 'uang', 'mungkin', 'tahan', 'beberapa', 'tahun', 'dapat', 'tangan', 'mental', 'terimakasih', 'sudah', 'buat'], ['aplikasi', 'sangat', 'bantu', 'aplikasi', 'banyak', 'ilmu', 'manfaat', 'tampil', 'aplikasi', 'tarik', 'banyak', 'pilih', 'pilih', 'nyaman', 'guna', 'aplikasi'], ['aplikasi', 'bantu', 'jaga', 'sehat', 'mental', 'baik', 'sehat', 'mental'], ['aplikasi', 'sangat', 'bagus', 'berterimakasiih', 'banyak', 'riliv', 'bantu', 'tani', 'masa', 'masa', 'sulit'], ['jauh', 'bantu', 'moga', 'selalu', 'tani', 'buat', 'sehat', 'mental', 'baik'], ['bagus', 'banget', 'banget', 'banget', 'semua', 'orang', 'ingin', 'bentuk', 'biasa', 'baru', 'utama', 'disiplin', 'diri', 'jaga', 'sehat', 'mental', 'butuh', 'ini', 'lebih', 'arah', 'terimakasih', 'buat', 'segal

# **Join**

In [17]:
# Join tokens back to string
data_preprocessed = [' '.join(tokens) for tokens in data_stemmed]
print(data_preprocessed)
#data_steammed berfungsi untuk mengubah data yang awalan array perkata menjadi satu kalimat

['sangat seru banyak journal journal keren bahasa aplikasi sangat ringan rasa sedang obrol teman', 'kira mental butuh tangan coba cari tangan aplikasi sadar tangan butuh uang mungkin tahan beberapa tahun dapat tangan mental terimakasih sudah buat', 'aplikasi sangat bantu aplikasi banyak ilmu manfaat tampil aplikasi tarik banyak pilih pilih nyaman guna aplikasi', 'aplikasi bantu jaga sehat mental baik sehat mental', 'aplikasi sangat bagus berterimakasiih banyak riliv bantu tani masa masa sulit', 'jauh bantu moga selalu tani buat sehat mental baik', 'bagus banget banget banget semua orang ingin bentuk biasa baru utama disiplin diri jaga sehat mental butuh ini lebih arah terimakasih buat segala fitur keren fungsional', 'fiturnya banyak aplikasi mudah paham tampil tarik banyak firtur sangat bantu banyak fitur eksplore hari bantu atas cemas diri', 'aplikasi bantu jalan hidup baik diri terima kasih riliv', 'aplikasi bagus sekali benar bantu urai kerumit kepala mungkin tingkat loading apps ce

# **Feature Extraction**

In [18]:
# Feature Extraction with TFIdf
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data_preprocessed)
vektor = X.toarray()
vektor.shape

y = dframe['score']

In [19]:
import pickle
pickle.dump(vectorizer, open("cv.pickle", "wb"))


%ls

 cv.pickle   ratingAll.csv       riliv-rating3.csv                         svm_model.pkl
 ds.pickle   reviews.pickle      riliv-rating5.csv
 kamus.csv   riliv-rating1.csv  'Sentimen Analisis Aplikasi Riliv.ipynb'


# **Modelling**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)  # tentukan jumlah tetangga (K)
knn.fit(vektor, y)

# Prediksi data uji
knn_y_pred = knn.predict(X_test)

# Evaluasi model
accuracy = accuracy_score(y_test, knn_y_pred)
print(f'Akurasi model KNN: {accuracy:.2f}')

report_knn = classification_report(y_test, knn_y_pred)
print ('report : ' +str(report_knn))

Akurasi model KNN: 0.69
report :               precision    recall  f1-score   support

           1       0.78      0.71      0.74        41
           3       0.54      0.78      0.64        36
           5       0.84      0.60      0.70        43

    accuracy                           0.69       120
   macro avg       0.72      0.70      0.69       120
weighted avg       0.73      0.69      0.70       120



## SVM

In [ ]:
from sklearn import svm

# Inisialisasi dan latih model SVM
svm = svm.SVC(kernel='linear')
svm.fit(vektor, y)

# Prediksi data uji
y_pred = svm.predict(X_test.toarray())

# Evaluasi model
accuracy = accuracy_score(y_test, y_pred)
print(f'Akurasi model SVM: {accuracy:.2f}')

report = classification_report(y_test, y_pred)
print ('report : ' +str(report))

Akurasi model SVM: 0.94
report :               precision    recall  f1-score   support

           1       0.91      1.00      0.95        41
           3       0.97      0.86      0.91        36
           5       0.95      0.95      0.95        43

    accuracy                           0.94       120
   macro avg       0.94      0.94      0.94       120
weighted avg       0.94      0.94      0.94       120



# Saving Model

Disini model terbaik menggunakan SVM jadi menggunakan model SVM dengan tingkat akurasi 0.78

In [ ]:
import pickle

# Save the model
with open('svm_model.pkl', 'wb') as file:
    pickle.dump(svm, file)

print("Model saved successfully.")

Model saved successfully.
